# In this notebook we will split barrel data into train and validation sets. They will be used during the training. We will also have an independent test set. The split is 60, 20, and 20% correspondingly.
### Start with imports and read in the barrel data.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython

In [2]:
import pandas as pd
import numpy as np
fileLocation = '/eos/user/r/rkamalie/'
barrel_df = pd.read_parquet(fileLocation + 'final_barrel_df.parquet')
# about 10 seconds

In [3]:
#memory usage and release 
#https://stackoverflow.com/questions/39100971/how-do-i-release-memory-used-by-a-pandas-dataframe

import os, psutil, numpy as np    
def usage():
    process = psutil.Process(os.getpid())
    return process.memory_info()[0] / float(2 ** 20)


In [4]:
# To print all columns of the dataframa when callling the dataftame or using its functions
def print_long_dataframe():
    pd.set_option('display.max_columns', None)  
    pd.set_option('display.expand_frame_repr', False)
    pd.set_option('max_colwidth', -1)
print_long_dataframe()

In [5]:
usage()

1622.9296875

In [6]:
ndf = barrel_df
ndf

,rho,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_iEtaOrX,sc_iPhiOrY,sc_iEtaMod5,sc_iPhiMod2,sc_iEtaMod20,sc_iPhiMod20,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ele_hademTow,ele_nrSatCrys,mc_energy,sc_seedClusEnergy_over_sc_rawEnergy,ssFull_e5x5_over_sc_rawEnergy,ssFull_e3x3_over_sc_rawEnergy,ssFull_eMax_over_ssFull_e5x5,ssFull_e2nd_over_ssFull_e5x5,ssFull_eTop_over_ssFull_e5x5,ssFull_eBottom_over_ssFull_e5x5,ssFull_eLeft_over_ssFull_e5x5,ssFull_eRight_over_ssFull_e5x5,ssFull_e2x5Max_over_ssFull_e5x5,ssFull_e2x5Left_over_ssFull_e5x5,ssFull_e2x5Right_over_ssFull_e5x5,ssFull_e2x5Top_over_ssFull_e5x5,ssFull_e2x5Bottom_over_ssFull_e5x5,meas_energy,Emc_over_Emeas
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,10.558399,96.798744,0.0,0.006568,0.031675,4.0,-5.031547e-04,-2.415152e-03,32.0,256.0,1.0,1.0,6.0,15.0,0.007908,0.065228,0.017998,0.000000,0.0,99.399963,0.728856,0.871792,0.728207,0.575223,0.111815,0.111815,0.040962,0.012127,0.044861,0.947412,0.042393,0.103093,0.269388,0.093580,96.798744,1.026872
1,10.558399,97.621788,0.0,0.009281,0.010019,1.0,0.000000e+00,0.000000e+00,-28.0,75.0,-2.0,0.0,-2.0,14.0,0.008893,-0.100080,0.009975,0.000000,0.0,99.399963,1.000000,1.006822,0.956975,0.319128,0.298260,0.298260,0.006052,0.004744,0.109407,0.963051,0.019996,0.345235,0.538833,0.019787,97.621788,1.018215
2,12.537903,221.094452,0.0,0.008480,0.012603,2.0,3.056435e-05,-5.005577e-04,-21.0,101.0,0.0,0.0,0.0,0.0,0.009775,0.009249,0.010154,0.000000,0.0,223.497986,0.996499,0.990720,0.954695,0.766905,0.068467,0.034249,0.030493,0.068467,0.016345,0.944445,0.117302,0.043192,0.075984,0.068648,221.094452,1.010871
3,12.537903,217.442139,0.0,0.010157,0.010345,1.0,0.000000e+00,0.000000e+00,22.0,280.0,1.0,1.0,1.0,19.0,0.009165,0.084857,0.010268,0.003535,0.0,223.497986,1.000000,0.993517,0.947777,0.610668,0.168521,0.074490,0.013213,0.168521,0.007896,0.951150,0.268452,0.026121,0.161701,0.043200,217.442139,1.027850
4,12.807579,189.944199,0.0,0.006963,0.030441,5.0,6.359640e-04,-1.577192e-04,-53.0,9.0,-2.0,0.0,-7.0,8.0,0.008288,0.046061,0.014327,0.000000,0.0,193.680542,0.956678,0.955256,0.876729,0.462956,0.334325,0.024942,0.334325,0.025031,0.019598,0.933934,0.063275,0.060414,0.051455,0.437529,189.944199,1.019671
5,5.446676,12.729020,0.0,0.007266,0.006614,1.0,0.000000e+00,-1.110223e-16,2.0,49.0,1.0,0.0,1.0,8.0,0.009273,-0.146166,0.009757,0.000000,0.0,14.144497,1.000000,1.061767,1.036085,0.652390,0.123199,0.010346,0.110834,0.123199,0.005216,0.971790,0.206575,0.019854,0.031615,0.187580,12.729020,1.111201
6,5.446676,12.650763,0.0,0.006147,0.008270,1.0,0.000000e+00,0.000000e+00,-1.0,223.0,0.0,0.0,0.0,2.0,0.010460,0.085742,0.009192,0.000000,0.0,14.144497,1.000000,1.087732,1.013172,0.534566,0.291393,0.007227,0.291393,0.017742,0.047092,0.936867,0.043557,0.113251,0.034574,0.354859,12.650763,1.118075
9,18.757620,432.131531,0.0,0.010091,0.016258,3.0,2.974516e-04,7.208383e-04,-57.0,341.0,-1.0,0.0,-11.0,0.0,0.008957,0.066931,0.010391,0.000000,0.0,434.072479,0.955292,0.961343,0.920282,0.472085,0.269646,0.110199,0.018438,0.269646,0.009782,0.955795,0.355106,0.031689,0.194351,0.048455,432.131531,1.004492
10,18.757620,380.564667,0.0,0.008509,0.018889,5.0,-3.158251e-04,-6.112282e-04,58.0,161.0,2.0,0.0,12.0,0.0,0.009559,-0.041916,0.011539,0.019697,0.0,434.072479,0.946917,0.953263,0.898939,0.592148,0.217226,0.011818,0.217226,0.041444,0.013689,0.941733,0.113431,0.043021,0.035665,0.310654,380.564667,1.140601


### Keep only variables important for training (32 for barrel and 29 for endcap) plus a target variables

In [7]:
barrel_vars = [
'Emc_over_Emeas',

    
'sc_rawEnergy',
'sc_etaWidth',
'sc_phiWidth',
'sc_seedClusEnergy_over_sc_rawEnergy',
'ssFull_e5x5_over_sc_rawEnergy',
'ele_hademTow',
'rho',
'sc_dEtaSeedSC',
'sc_dPhiSeedSC',
'ssFull_e3x3_over_sc_rawEnergy',
'ssFull_sigmaIEtaIEta',
'ssFull_sigmaIEtaIPhi',
'ssFull_sigmaIPhiIPhi',
'ssFull_eMax_over_ssFull_e5x5',
'ssFull_e2nd_over_ssFull_e5x5',
'ssFull_eTop_over_ssFull_e5x5',
'ssFull_eBottom_over_ssFull_e5x5',
'ssFull_eLeft_over_ssFull_e5x5',
'ssFull_eRight_over_ssFull_e5x5',
'ssFull_e2x5Max_over_ssFull_e5x5',
'ssFull_e2x5Left_over_ssFull_e5x5',
'ssFull_e2x5Right_over_ssFull_e5x5',
'ssFull_e2x5Top_over_ssFull_e5x5',
'ssFull_e2x5Bottom_over_ssFull_e5x5',
'ele_nrSatCrys',
'sc_numberOfClusters',
'sc_iEtaOrX',
'sc_iPhiOrY',
'sc_iEtaMod5',
'sc_iPhiMod2',
'sc_iEtaMod20',
'sc_iPhiMod20'
]
len(barrel_vars)

33

In [8]:
endcap_vars = [

'Emc_over_Emeas',
    
'sc_rawEnergy',
'sc_etaWidth',
'sc_phiWidth',
'sc_seedClusEnergy_over_sc_rawEnergy',
'ssFull_e5x5_over_sc_rawEnergy',
'ele_hademTow',
'rho',
'sc_dEtaSeedSC',
'sc_dPhiSeedSC',
'ssFull_e3x3_over_sc_rawEnergy',
'ssFull_sigmaIEtaIEta',
'ssFull_sigmaIEtaIPhi',
'ssFull_sigmaIPhiIPhi',
'ssFull_eMax_over_ssFull_e5x5',
'ssFull_e2nd_over_ssFull_e5x5',
'ssFull_eTop_over_ssFull_e5x5',
'ssFull_eBottom_over_ssFull_e5x5',
'ssFull_eLeft_over_ssFull_e5x5',
'ssFull_eRight_over_ssFull_e5x5',
'ssFull_e2x5Max_over_ssFull_e5x5',
'ssFull_e2x5Left_over_ssFull_e5x5',
'ssFull_e2x5Right_over_ssFull_e5x5',
'ssFull_e2x5Top_over_ssFull_e5x5',
'ssFull_e2x5Bottom_over_ssFull_e5x5',
'ele_nrSatCrys',
'sc_numberOfClusters',
'sc_iEtaOrX',
'sc_iPhiOrY',
'sc_rawESEnergy_over_sc_rawEnergy'
    
]
len(endcap_vars)

30

In [8]:
ndf = ndf[ndf.columns & barrel_vars]
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8604800 entries, 0 to 14985167
Data columns (total 33 columns):
rho                                    float32
sc_rawEnergy                           float32
sc_etaWidth                            float32
sc_phiWidth                            float32
sc_numberOfClusters                    float32
sc_dEtaSeedSC                          float32
sc_dPhiSeedSC                          float32
sc_iEtaOrX                             float32
sc_iPhiOrY                             float32
sc_iEtaMod5                            float32
sc_iPhiMod2                            float32
sc_iEtaMod20                           float32
sc_iPhiMod20                           float32
ssFull_sigmaIEtaIEta                   float32
ssFull_sigmaIEtaIPhi                   float32
ssFull_sigmaIPhiIPhi                   float32
ele_hademTow                           float32
ele_nrSatCrys                          float32
sc_seedClusEnergy_over_sc_rawEnergy    f

### Now split dataframe into 'x' and 'y' sets. The 'x' set represents a set of features or variables, call it 'data'. The 'y' set is just a single column which represents the ratio of electron energies of interest, call it a target.

In [3]:
data = ndf.drop('Emc_over_Emeas', axis=1)
target = ndf.loc[:, ['Emc_over_Emeas']]
target.info()

NameError: name 'ndf' is not defined

In [10]:
target.describe()

,Emc_over_Emeas
count,8.604800e+06
mean,1.078456e+00
std,1.258465e+00
min,1.287299e-01
25%,1.005088e+00
50%,1.017478e+00
75%,1.044554e+00
max,7.114703e+02


In [11]:
data

,rho,sc_rawEnergy,sc_etaWidth,sc_phiWidth,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_iEtaOrX,sc_iPhiOrY,sc_iEtaMod5,sc_iPhiMod2,sc_iEtaMod20,sc_iPhiMod20,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ele_hademTow,ele_nrSatCrys,sc_seedClusEnergy_over_sc_rawEnergy,ssFull_e5x5_over_sc_rawEnergy,ssFull_e3x3_over_sc_rawEnergy,ssFull_eMax_over_ssFull_e5x5,ssFull_e2nd_over_ssFull_e5x5,ssFull_eTop_over_ssFull_e5x5,ssFull_eBottom_over_ssFull_e5x5,ssFull_eLeft_over_ssFull_e5x5,ssFull_eRight_over_ssFull_e5x5,ssFull_e2x5Max_over_ssFull_e5x5,ssFull_e2x5Left_over_ssFull_e5x5,ssFull_e2x5Right_over_ssFull_e5x5,ssFull_e2x5Top_over_ssFull_e5x5,ssFull_e2x5Bottom_over_ssFull_e5x5
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,10.558399,96.798744,0.006568,0.031675,4.0,-5.031547e-04,-2.415152e-03,32.0,256.0,1.0,1.0,6.0,15.0,0.007908,0.065228,0.017998,0.000000,0.0,0.728856,0.871792,0.728207,0.575223,0.111815,0.111815,0.040962,0.012127,0.044861,0.947412,0.042393,0.103093,0.269388,0.093580
1,10.558399,97.621788,0.009281,0.010019,1.0,0.000000e+00,0.000000e+00,-28.0,75.0,-2.0,0.0,-2.0,14.0,0.008893,-0.100080,0.009975,0.000000,0.0,1.000000,1.006822,0.956975,0.319128,0.298260,0.298260,0.006052,0.004744,0.109407,0.963051,0.019996,0.345235,0.538833,0.019787
2,12.537903,221.094452,0.008480,0.012603,2.0,3.056435e-05,-5.005577e-04,-21.0,101.0,0.0,0.0,0.0,0.0,0.009775,0.009249,0.010154,0.000000,0.0,0.996499,0.990720,0.954695,0.766905,0.068467,0.034249,0.030493,0.068467,0.016345,0.944445,0.117302,0.043192,0.075984,0.068648
3,12.537903,217.442139,0.010157,0.010345,1.0,0.000000e+00,0.000000e+00,22.0,280.0,1.0,1.0,1.0,19.0,0.009165,0.084857,0.010268,0.003535,0.0,1.000000,0.993517,0.947777,0.610668,0.168521,0.074490,0.013213,0.168521,0.007896,0.951150,0.268452,0.026121,0.161701,0.043200
4,12.807579,189.944199,0.006963,0.030441,5.0,6.359640e-04,-1.577192e-04,-53.0,9.0,-2.0,0.0,-7.0,8.0,0.008288,0.046061,0.014327,0.000000,0.0,0.956678,0.955256,0.876729,0.462956,0.334325,0.024942,0.334325,0.025031,0.019598,0.933934,0.063275,0.060414,0.051455,0.437529
5,5.446676,12.729020,0.007266,0.006614,1.0,0.000000e+00,-1.110223e-16,2.0,49.0,1.0,0.0,1.0,8.0,0.009273,-0.146166,0.009757,0.000000,0.0,1.000000,1.061767,1.036085,0.652390,0.123199,0.010346,0.110834,0.123199,0.005216,0.971790,0.206575,0.019854,0.031615,0.187580
6,5.446676,12.650763,0.006147,0.008270,1.0,0.000000e+00,0.000000e+00,-1.0,223.0,0.0,0.0,0.0,2.0,0.010460,0.085742,0.009192,0.000000,0.0,1.000000,1.087732,1.013172,0.534566,0.291393,0.007227,0.291393,0.017742,0.047092,0.936867,0.043557,0.113251,0.034574,0.354859
9,18.757620,432.131531,0.010091,0.016258,3.0,2.974516e-04,7.208383e-04,-57.0,341.0,-1.0,0.0,-11.0,0.0,0.008957,0.066931,0.010391,0.000000,0.0,0.955292,0.961343,0.920282,0.472085,0.269646,0.110199,0.018438,0.269646,0.009782,0.955795,0.355106,0.031689,0.194351,0.048455
10,18.757620,380.564667,0.008509,0.018889,5.0,-3.158251e-04,-6.112282e-04,58.0,161.0,2.0,0.0,12.0,0.0,0.009559,-0.041916,0.011539,0.019697,0.0,0.946917,0.953263,0.898939,0.592148,0.217226,0.011818,0.217226,0.041444,0.013689,0.941733,0.113431,0.043021,0.035665,0.310654


from sklearn.preprocessing import MinMaxScaler
useScaler = True

if useScaler:
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    print(scaler_x.fit(data))
    x_scaled=scaler_x.fit_transform(data)
    print(scaler_y.fit(target))
    y_scaled=scaler_y.fit_transform(target)
else:
    x_scaled = data
    y_scaled = target
    

In [12]:
usage()

3763.09375

### Split the data. Notice, since we are splitting into 3 partitions, we need to call the method twice.

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=1)

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.25, random_state=1)

#In this way, train, val, test set will be 60%, 20%, 20% of the dataset respectively.


In [14]:
x_train.shape


(5162880, 32)

In [15]:
x_test.shape


(1720960, 32)

In [16]:
x_val.shape

(1720960, 32)

In [17]:
y_val.shape

(1720960, 1)

In [23]:
x_train

,rho,sc_rawEnergy,sc_etaWidth,sc_phiWidth,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_iEtaOrX,sc_iPhiOrY,sc_iEtaMod5,sc_iPhiMod2,sc_iEtaMod20,sc_iPhiMod20,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ele_hademTow,ele_nrSatCrys,sc_seedClusEnergy_over_sc_rawEnergy,ssFull_e5x5_over_sc_rawEnergy,ssFull_e3x3_over_sc_rawEnergy,ssFull_eMax_over_ssFull_e5x5,ssFull_e2nd_over_ssFull_e5x5,ssFull_eTop_over_ssFull_e5x5,ssFull_eBottom_over_ssFull_e5x5,ssFull_eLeft_over_ssFull_e5x5,ssFull_eRight_over_ssFull_e5x5,ssFull_e2x5Max_over_ssFull_e5x5,ssFull_e2x5Left_over_ssFull_e5x5,ssFull_e2x5Right_over_ssFull_e5x5,ssFull_e2x5Top_over_ssFull_e5x5,ssFull_e2x5Bottom_over_ssFull_e5x5
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14160264,2.250513,338.077942,0.007222,0.011672,2.0,-0.000040,-5.702713e-04,34.0,164.0,3.0,1.0,8.0,3.0,0.008931,0.137189,0.010642,0.003183,0.0,0.994657,0.989897,0.953875,0.779164,0.058043,0.024814,0.058043,0.015575,0.042118,0.948637,0.043235,0.081756,0.051641,0.106064
4940382,34.319759,255.862213,0.010254,0.040728,6.0,-0.001393,-2.404200e-03,69.0,352.0,3.0,1.0,3.0,11.0,0.009082,0.077053,0.011861,0.000000,0.0,0.972759,0.971470,0.932001,0.665406,0.096582,0.042180,0.084574,0.096582,0.009640,0.958279,0.172906,0.025542,0.093612,0.129166
5598260,40.743256,34.173145,0.008002,0.006858,1.0,0.000000,0.000000e+00,8.0,80.0,2.0,1.0,7.0,19.0,0.009149,0.019079,0.010565,0.000000,0.0,1.000000,1.031125,0.993001,0.703480,0.140046,0.033913,0.026029,0.140046,0.004793,0.961113,0.210974,0.018136,0.079656,0.063745
922992,0.000000,302.637299,0.007001,0.014635,3.0,0.000079,-1.523724e-04,-75.0,71.0,-4.0,0.0,-9.0,10.0,0.008527,-0.068100,0.014646,0.000000,0.0,0.998167,0.984724,0.930243,0.716379,0.104450,0.104450,0.030532,0.018109,0.028651,0.945402,0.046967,0.071102,0.157903,0.074678
3265009,29.131996,143.315887,0.006467,0.013441,2.0,-0.000155,2.303528e-03,12.0,146.0,1.0,1.0,11.0,5.0,0.008822,-0.003132,0.011031,0.000000,0.0,0.947930,0.962919,0.929619,0.702238,0.152290,0.022384,0.152290,0.021884,0.021524,0.938920,0.057502,0.053348,0.042268,0.206619
9222065,36.376659,229.838379,0.009594,0.016597,2.0,0.000076,-3.878572e-04,-41.0,74.0,0.0,1.0,-15.0,13.0,0.009173,-0.104456,0.009665,0.000000,0.0,0.998986,0.996310,0.957809,0.433307,0.176899,0.011409,0.160327,0.176899,0.010260,0.958666,0.357464,0.026911,0.032958,0.341519
8317177,0.000000,249.337524,0.007463,0.017684,4.0,0.000176,1.730705e-03,-28.0,199.0,-2.0,0.0,-2.0,18.0,0.008310,-0.117097,0.012605,0.000000,0.0,0.937040,0.952360,0.902906,0.555515,0.265914,0.265914,0.027278,0.010143,0.033628,0.951052,0.036958,0.094670,0.337402,0.058093
14314885,24.143108,139.754379,0.006825,0.009840,1.0,0.000000,0.000000e+00,-12.0,310.0,-1.0,1.0,-11.0,9.0,0.009000,-0.013248,0.011783,0.000000,0.0,1.000000,1.006182,0.971487,0.632908,0.187387,0.187387,0.040400,0.037063,0.013363,0.947026,0.083030,0.046387,0.244287,0.067923
9439796,38.691669,118.509178,0.008249,0.043747,3.0,0.001427,-2.356774e-04,-71.0,304.0,0.0,1.0,-5.0,3.0,0.006593,0.013258,0.014567,0.012324,0.0,0.986417,0.983465,0.925071,0.604771,0.229778,0.051918,0.229778,0.000000,0.023848,0.978868,0.010265,0.074029,0.089517,0.280044


### It took quite a while to reach this point, let's save our train, validation, and test sets for both data and target. Then, reading it in a new notebook will be a matter of 5 seconds.

In [24]:
np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)

np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)

np.save('x_val.npy', x_val)
np.save('y_val.npy', y_val)

#array_reloaded = np.load('fnumpy.npy')